In [1]:
!pip install kaggle

In [2]:
# configuring the path od Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# changing the permission of the file
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [3]:
#Api to fetch dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 82% 66.0M/80.9M [00:00<00:00, 152MB/s]
100% 80.9M/80.9M [00:00<00:00, 151MB/s]


In [4]:
#extracting the comprosed dataset
from zipfile import ZipFile
file_name = "/content/sentiment140.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Done")

Done


In [5]:
import pandas as pd
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [6]:
#naming the columns and reading dataset again because that first columns contains data point not feature we are changing this
column_names = ['target','id','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding = 'ISO-8859-1')

In [7]:
twitter_data.shape
#column names are added +1

(1600000, 6)

In [8]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
twitter_data["target"].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [10]:
#Convert the target(positive tweets) "4" to "1"
twitter_data.loc[twitter_data['target'] == 4, 'target'] = 1


In [11]:
twitter_data["target"].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [12]:

# Drop unnecessary columns
twitter_data.drop(['id', 'date', 'flag', 'user'], axis=1, inplace=True)

In [13]:
# Display the first few rows to confirm changes
twitter_data.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [14]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(twitter_data, test_size=0.2, random_state=42, stratify=twitter_data['target'])

In [15]:
# Check for NaN values
print(train_df.isna().sum())
print(test_df.isna().sum())

target    0
text      0
dtype: int64
target    0
text      0
dtype: int64


In [16]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [17]:
import tensorflow as tf

# Convert the features and labels into TensorFlow datasets
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dataframe['text'].values, labels.values))
    if shuffle:
        ds = ds.shuffle(buffer_size=10000)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

# Convert DataFrames to TensorFlow datasets
train_dataset = df_to_dataset(train_df, shuffle=True, batch_size=64)
test_dataset = df_to_dataset(test_df, shuffle=False, batch_size=64)


In [18]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b"@risager found any good documentation on mailpress?  I can't get the form() arguments to work  #wordpress"
 b'@radio2xs thanks guys ' b'I hate June gloom! Workin at 2 today ']

labels:  [0 1 1]


In [19]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [20]:
import numpy as np
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'i', 'to', 'the', 'a', 'my', 'and', 'you', 'is', 'it',
       'for', 'in', 'of', 'im', 'on', 'me', 'so', 'have', 'that'],
      dtype='<U13')

In [21]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[  1, 313, 184,  30,   1,  15,   1,   2,  48,  33,   4,   1,   1,
          3,  47,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  1,  76, 198,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  2, 147, 561,   1,   1,  24,  81,  43,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0]])

In [22]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b"@risager found any good documentation on mailpress?  I can't get the form() arguments to work  #wordpress"
Round-trip:  [UNK] found any good [UNK] on [UNK] i cant get the [UNK] [UNK] to work [UNK]             

Original:  b'@radio2xs thanks guys '
Round-trip:  [UNK] thanks guys                          

Original:  b'I hate June gloom! Workin at 2 today '
Round-trip:  i hate june [UNK] [UNK] at 2 today                     



In [34]:
lstm_model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [35]:
lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, None)              0         
 ctorization)                                                    
                                                                 
 embedding_2 (Embedding)     (None, None, 64)          64000     
                                                                 
 bidirectional_2 (Bidirecti  (None, 128)               66048     
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 138369 (540.50 KB)
Trainable params: 138

In [4]:
!pip install tensorflow-addons


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 14.5 MB/s eta 0:00:00


In [36]:
import tensorflow_addons as tfa
lstm_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=[
        tf.keras.metrics.MeanSquaredError(),
        tf.keras.metrics.AUC(),
        tf.keras.metrics.BinaryAccuracy(),
        tfa.metrics.F1Score(num_classes=1, average='macro', threshold=0.5)  # F1 skoru için
    ]
)

In [37]:
history = lstm_model.fit(
    train_dataset,
    epochs=5,
    validation_data=test_dataset,
    validation_steps=30
)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5818: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


20000/20000 [==============================] - 249s 12ms/step - loss: 0.4822 - mean_squared_error: 0.1589 - auc_2: 0.8482 - binary_accuracy: 0.7668 - f1_score: 0.7671 - val_loss: 0.4819 - val_mean_squared_error: 0.1589 - val_auc_2: 0.8491 - val_binary_accuracy: 0.7646 - val_f1_score: 0.7703
Epoch 2/5
20000/20000 [==============================] - 228s 11ms/step - loss: 0.4598 - mean_squared_error: 0.1507 - auc_2: 0.8632 - binary_accuracy: 0.7788 - f1_score: 0.7797 - val_loss: 0.4690 - val_mean_squared_error: 0.1543 - val_auc_2: 0.8570 - val_binary_accuracy: 0.7729 - val_f1_score: 0.7796
Epoch 3/5
20000/20000 [==============================] - 227s 11ms/step - loss: 0.4489 - mean_squared_error: 0.1466 - auc_2: 0.8703 - binary_accuracy: 0.7855 - f1_score: 0.7871 - val_loss: 0.4610 - val_mean_squared_error: 0.1512 - val_auc_2: 0.8637 - val_binary_accuracy: 0.7807 - val_f1_score: 0.7849
Epoch 4/5
20000/20000 [==============================] - 227s 11ms/step - loss: 0.4420 - mean_squared_er

In [38]:
test_results = lstm_model.evaluate(test_dataset)
print(f"Test Results - Loss: {test_results[0]}, MSE: {test_results[1]}, AUC: {test_results[2]}, Accuracy: {test_results[3]}, F1 Score: {test_results[4]}")

5000/5000 [==============================] - 26s 5ms/step - loss: 0.4379 - mean_squared_error: 0.1427 - auc_2: 0.8772 - binary_accuracy: 0.7914 - f1_score: 0.7944
Test Results - Loss: 0.43785589933395386, MSE: 0.1426505148410797, AUC: 0.8771782517433167, Accuracy: 0.7913812398910522, F1 Score: 0.794430136680603


In [40]:
import numpy as np

sample_text = np.array(["thanks guys", "i hate you", "this was bad"])

# if the prediction is >= 0.5, it is positive else it is negative
predictions = lstm_model.predict(sample_text)
predictions


1/1 [==============================] - 2s 2s/step


array([[0.98075   ],
       [0.04823587],
       [0.08379021]], dtype=float32)

In [41]:
lstm_model.save('lstm_model_twitter.keras', save_format='keras')


In [5]:
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa

model_loaded = load_model('lstm_model_twitter.keras')

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [6]:
import numpy as np

sample_text = np.array(["thanks guys", "i hate you", "this was bad"])

# if the prediction is >= 0.5, it is positive else it is negative
predictions = model_loaded.predict(sample_text)
predictions


1/1 [==============================] - 4s 4s/step


array([[0.98075   ],
       [0.04823587],
       [0.08379021]], dtype=float32)